In [1]:
import torch, torchvision
from torchvision import datasets, models, transforms
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

from torch.autograd import Variable
import torchvision


import time
from torchsummary import summary
import pathlib

import numpy as np
import matplotlib.pyplot as plt
import os

from PIL import Image

In [2]:
# Load the Data

data_dir = pathlib.Path('../data/tiny-imagenet-200')
image_count = len(list(data_dir.glob('**/*.JPEG')))
CLASS_NAMES = np.array([item.name for item in (data_dir / 'train').glob('*')])
num_classes = len(CLASS_NAMES)
print('Discovered {} images in {} classes'.format(image_count, num_classes))

# Create the training data generator
batch_size = 32
im_height = 64
im_width = 64
num_epochs = 1

data_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0, 0, 0), tuple(np.sqrt((255, 255, 255)))),
])

# Load Data from folders
data = {
    'train': datasets.ImageFolder(root=data_dir / 'train', transform=data_transforms),
    'valid': datasets.ImageFolder(root=data_dir / 'val', transform=data_transforms),
    'test': datasets.ImageFolder(root=data_dir / 'test', transform=data_transforms)
}

# Get a mapping of the indices to the class names, in order to see the output classes of the test images.
idx_to_class = {v: k for k, v in data['train'].class_to_idx.items()}
print(idx_to_class)

# Size of Data, to be used for calculating Average Loss and Accuracy
train_data_size = len(data['train'])
valid_data_size = len(data['valid'])
test_data_size = len(data['test'])

# Create iterators for the Data loaded using DataLoader module
train_data_loader = DataLoader(data['train'], batch_size=batch_size, shuffle=True)
valid_data_loader = DataLoader(data['valid'], batch_size=batch_size, shuffle=True)
test_data_loader = DataLoader(data['test'], batch_size=batch_size, shuffle=True)





# train_set = torchvision.datasets.ImageFolder(data_dir / 'train', data_transforms)
# train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size,
#                                            shuffle=True, num_workers=4, pin_memory=True)



# # Set train and valid directory paths
# train_directory = os.path.join(dataset, 'train')
# valid_directory = os.path.join(dataset, 'valid')
# test_directory = os.path.join(dataset, 'test')

# # Batch size
# bs = 32

# # Number of classes
# num_classes = len(os.listdir(valid_directory))-1  #10#2#257
# print(num_classes)

# # Load Data from folders
# data = {
#     'train': datasets.ImageFolder(root=train_directory, transform=image_transforms['train']),
#     'valid': datasets.ImageFolder(root=valid_directory, transform=image_transforms['valid']),
#     'test': datasets.ImageFolder(root=test_directory, transform=image_transforms['test'])
# }

# # Get a mapping of the indices to the class names, in order to see the output classes of the test images.
# idx_to_class = {v: k for k, v in data['train'].class_to_idx.items()}
# print(idx_to_class)

# # Size of Data, to be used for calculating Average Loss and Accuracy
# train_data_size = len(data['train'])
# valid_data_size = len(data['valid'])
# test_data_size = len(data['test'])

# # Create iterators for the Data loaded using DataLoader module
# train_data_loader = DataLoader(data['train'], batch_size=bs, shuffle=True)
# valid_data_loader = DataLoader(data['valid'], batch_size=bs, shuffle=True)
# test_data_loader = DataLoader(data['test'], batch_size=bs, shuffle=True)

Discovered 129998 images in 200 classes
{0: 'n01443537', 1: 'n01629819', 2: 'n01641577', 3: 'n01644900', 4: 'n01698640', 5: 'n01742172', 6: 'n01768244', 7: 'n01770393', 8: 'n01774384', 9: 'n01774750', 10: 'n01784675', 11: 'n01855672', 12: 'n01882714', 13: 'n01910747', 14: 'n01917289', 15: 'n01944390', 16: 'n01945685', 17: 'n01950731', 18: 'n01983481', 19: 'n01984695', 20: 'n02002724', 21: 'n02056570', 22: 'n02058221', 23: 'n02074367', 24: 'n02085620', 25: 'n02094433', 26: 'n02099601', 27: 'n02099712', 28: 'n02106662', 29: 'n02113799', 30: 'n02123045', 31: 'n02123394', 32: 'n02124075', 33: 'n02125311', 34: 'n02129165', 35: 'n02132136', 36: 'n02165456', 37: 'n02190166', 38: 'n02206856', 39: 'n02226429', 40: 'n02231487', 41: 'n02233338', 42: 'n02236044', 43: 'n02268443', 44: 'n02279972', 45: 'n02281406', 46: 'n02321529', 47: 'n02364673', 48: 'n02395406', 49: 'n02403003', 50: 'n02410509', 51: 'n02415577', 52: 'n02423022', 53: 'n02437312', 54: 'n02480495', 55: 'n02481823', 56: 'n02486410', 

In [3]:
train_data_size, valid_data_size, test_data_size

(100000, 9999, 10000)

In [6]:
# Load pretrained ResNet50 Model
model = models.resnet50(pretrained=True)
model = model.to('cuda:0')

In [7]:
# Freeze model parameters
for param in model.parameters():
    param.requires_grad = False
    
# Change the final layer of ResNet50 Model for Transfer Learning
fc_inputs = model.fc.in_features

model.fc = nn.Sequential(
    nn.Linear(fc_inputs, 256),
    nn.ReLU(),
    nn.Dropout(0.4),
    nn.Linear(256, num_classes), # Since 10 possible outputs
    nn.LogSoftmax(dim=1) # For using NLLLoss()
)

# Convert model to be used on GPU
model = model.to('cuda:0')

# Define Optimizer and Loss Function
# loss_func = nn.NLLLoss()
loss_func = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

In [30]:
# Load pretrained ResNet50 Model
model = models.resnet50(pretrained=True)
model = model.to('cuda:0')

In [31]:
# Freeze model parameters
for param in model.parameters():
    param.requires_grad = False

    
# Change the final layer of ResNet50 Model for Transfer Learning
fc_inputs = model.fc.in_features

model.fc = nn.Sequential(
    nn.Linear(fc_inputs, 256),
    nn.ReLU(),
    nn.Dropout(0.4),
    nn.Linear(256, num_classes), # Since 10 possible outputs
    nn.LogSoftmax(dim=1) # For using NLLLoss()
)

# Convert model to be used on GPU
model = model.to('cuda:0')

# Define Optimizer and Loss Function
# loss_func = nn.NLLLoss()
loss_func = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

In [35]:
import sys
sys.path.append("..")

In [36]:
!python train_model.py -data "../data/tiny-imagenet-200/train/" -a "resnet50" --epochs 10 --pretrained --gpu 0

OSError: [Errno 12] Cannot allocate memory

In [8]:
# Change the final layer of the box predictor for transfer learning
box_predictor_in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor.cls_score = nn.Linear(in_features=1024, out_features=num_classes, bias=True)

# Change the final layer of the mask predictor for transfer learning
mask_logits = model.roi_heads.mask_predictor.mask_fcn_logits
mask_logits_in_channels = mask_logits.in_channels
mask_logits_kernel_size = mask_logits.kernel_size
mask_logits_stride = mask_logits.stride
model.roi_heads.mask_predictor.mask_fcn_logits = nn.Conv2d(mask_logits_in_channels, 
                                                          num_classes,
                                                          kernel_size=mask_logits_kernel_size,
                                                          stride=mask_logits_stride)

# Convert model to be used on GPU
model = model.to('cuda:0')

In [9]:
optimizer = torch.optim.Adam(model.parameters())
loss_func = nn.CrossEntropyLoss()

In [13]:
def validate_only(model, loss_criterion, data_loader):
    '''
    Function to train and validate
    Parameters
        :param model: Model to train and validate
        :param loss_criterion: Loss Criterion to minimize
        :param optimizer: Optimizer for computing gradients
        :param epochs: Number of epochs (default=25)
  
    Returns
        model: Trained Model with best validation accuracy
        history: (dict object): Having training loss, accuracy and validation loss, accuracy
    '''
    
    start = time.time()
    history = []
    best_acc = 0.0
    
    print_every = 50

    epoch_start = time.time()

    # Set to training mode
    model.eval()

    # Loss and Accuracy within the epoch
    train_loss = 0.0
    train_acc = 0.0

    valid_loss = 0.0
    valid_acc = 0.0

    # Validation - No gradient tracking needed
    with torch.no_grad():
        # Set to evaluation mode
        model.eval()

        # Validation loop
        for j, (inputs, labels) in enumerate(data_loader):
            inputs = inputs.to(device)
            labels = labels.to(device)

            # Forward pass - compute outputs on input data using the model
            outputs = model(inputs)
            # print(outputs.shape)
            loss = loss_criterion(outputs, labels)
            valid_loss += loss.item() * inputs.size(0)

            # Calculate validation accuracy
            ret, predictions = torch.max(outputs.data, 1)
            correct_counts = predictions.eq(labels.data.view_as(predictions))

            # Convert correct_counts to float and then compute the mean
            acc = torch.mean(correct_counts.type(torch.FloatTensor))

            # Compute total accuracy in the whole batch and add to valid_acc
            valid_acc += acc.item() * inputs.size(0)

            #print("Validation Batch number: {:03d}, Validation: Loss: {:.4f}, Accuracy: {:.4f}".format(j, loss.item(), acc.item()))

        # Find average training loss and training accuracy
        avg_valid_loss = valid_loss/valid_data_size 
        avg_valid_acc = valid_acc/valid_data_size
                
        epoch_end = time.time()
    
        print("Validation : Loss : {:.4f}, Accuracy: {:.4f}%, Time: {:.4f}s".format(avg_valid_loss, avg_valid_acc*100, epoch_end-epoch_start))
        return avg_valid_acc, avg_valid_loss

In [9]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = torch.load('../models/model_2.pt')

In [10]:
def test_transform_combo(transforms_combo):
    data_transforms = transforms.Compose(transforms_combo)

    # Load Data from folders
    test_train_data = datasets.ImageFolder(root=data_dir / 'val', transform=data_transforms)

    # Size of Data, to be used for calculating Average Loss and Accuracy
    test_train_data_size = len(test_train_data)

    # Create iterators for the Data loaded using DataLoader module
    test_train_data_loader = DataLoader(test_train_data, batch_size=batch_size, shuffle=True)
    
    avg_valid_acc, avg_valid_loss = validate_only(model, loss_func, test_train_data_loader)
    return avg_valid_acc, avg_valid_loss

In [11]:
methods = []
accuracies = []

# Freeze model parameters
for param in model.parameters():
    param.requires_grad = False

In [14]:
transforms_combo = [
        transforms.ToTensor(),
        transforms.Normalize((0, 0, 0), tuple(np.sqrt((255, 255, 255)))),
    ]

avg_valid_acc, avg_valid_loss = test_transform_combo(transforms_combo)
methods.append("Default")
accuracies.append(avg_valid_acc)

Validation : Loss : 4.4746, Accuracy: 7.8508%, Time: 62.9266s


In [36]:
transforms_combo = [
        transforms.ToTensor(),
        transforms.Normalize((0, 0, 0), tuple(np.sqrt((255, 255, 255)))),
    ]

avg_valid_acc, avg_valid_loss = test_transform_combo(transforms_combo)
methods.append("Default")
accuracies.append(avg_valid_acc)

Validation : Loss : 6.6755, Accuracy: 0.8500%, Time: 15.1313s


In [46]:
def make_fooling_image(X, target_y, model):
    """
    Generate a fooling image that is close to X, but that the model classifies
    as target_y.

    Inputs:
    - X: Input image; Tensor of shape (1, 3, 224, 224)
    - target_y: An integer in the range [0, 1000)
    - model: A pretrained CNN

    Returns:
    - X_fooling: An image that is close to X, but that is classifed as target_y
    by the model.
    """
    # Initialize our fooling image to the input image, and wrap it in a Variable.
    X_fooling = X.clone()
    X_fooling_var = Variable(X_fooling, requires_grad=True)
    
    learning_rate = 1
    ##############################################################################
    # TODO: Generate a fooling image X_fooling that the model will classify as   #
    # the class target_y. You should perform gradient ascent on the score of the #
    # target class, stopping when the model is fooled.                           #
    # When computing an update step, first normalize the gradient:               #
    #   dX = learning_rate * g / ||g||_2                                         #
    #                                                                            #
    # You should write a training loop.                                          #
    #                                                                            #
    # HINT: For most examples, you should be able to generate a fooling image    #
    # in fewer than 100 iterations of gradient ascent.                           #
    # You can print your progress over iterations to check your algorithm.       #
    ##############################################################################
    fooled = False
    itr = 1
    
    while not fooled:
        scores = model(X_fooling_var)  # Run forward propagation
        print(scores.shape)
        
        # Check if classified image == target image
        _, i = torch.max(scores, dim=1)
        if i.item() == target_y:
            fooled = True
        else:
            scores_target = scores[0, target_y]  # Track the loss for the correct classes
            scores_target.backward()  # Run backward propagation

            # Update fooling
            X_fooling_var.data += learning_rate * X_fooling_var.grad.data / torch.norm(X_fooling_var.grad.data)
            X_fooling_var.grad.data.zero_()  # Office hours directed me to this page: https://www.quora.com/Why-do-we-need-to-set-the-gradients-manually-to-zero-in-Pytorch
            
        itr += 1
        if itr % 5 == 0:
            print('Iter', itr)
    ##############################################################################
    #                             END OF YOUR CODE                               #
    ##############################################################################
    print('# of Iter', itr)
    return X_fooling

fooling_transform = lambda x: make_fooling_image(x, 0, model)

In [51]:
def test_transform_combo(transforms_combo):
    data_transforms = transforms.Compose(transforms_combo)

    # Load Data from folders
    test_train_data = datasets.ImageFolder(root=data_dir / 'val', transform=data_transforms)

    # Size of Data, to be used for calculating Average Loss and Accuracy
    test_train_data_size = len(test_train_data)

    # Create iterators for the Data loaded using DataLoader module
    test_train_data_loader = DataLoader(test_train_data, batch_size=1, shuffle=True)
    
    avg_valid_acc, avg_valid_loss = validate_only(model, loss_func, test_train_data_loader)
    return avg_valid_acc, avg_valid_loss

transforms_combo = [
        transforms.ToTensor(),
        transforms.Normalize((0, 0, 0), tuple(np.sqrt((255, 255, 255))))
    ]

avg_valid_acc, avg_valid_loss = test_transform_combo(transforms_combo)
methods.append("Fooling Images")
accuracies.append(avg_valid_acc)

torch.Size([1, 200])
torch.Size([1, 200])
torch.Size([1, 200])
torch.Size([1, 200])
torch.Size([1, 200])
torch.Size([1, 200])
torch.Size([1, 200])
torch.Size([1, 200])
torch.Size([1, 200])
torch.Size([1, 200])
torch.Size([1, 200])
torch.Size([1, 200])
torch.Size([1, 200])
torch.Size([1, 200])
torch.Size([1, 200])
torch.Size([1, 200])
torch.Size([1, 200])
torch.Size([1, 200])
torch.Size([1, 200])
torch.Size([1, 200])
torch.Size([1, 200])
torch.Size([1, 200])
torch.Size([1, 200])
torch.Size([1, 200])
torch.Size([1, 200])
torch.Size([1, 200])
torch.Size([1, 200])
torch.Size([1, 200])
torch.Size([1, 200])
torch.Size([1, 200])
torch.Size([1, 200])
torch.Size([1, 200])
torch.Size([1, 200])
torch.Size([1, 200])
torch.Size([1, 200])
torch.Size([1, 200])
torch.Size([1, 200])
torch.Size([1, 200])
torch.Size([1, 200])
torch.Size([1, 200])
torch.Size([1, 200])
torch.Size([1, 200])
torch.Size([1, 200])
torch.Size([1, 200])
torch.Size([1, 200])
torch.Size([1, 200])
torch.Size([1, 200])
torch.Size([1

KeyboardInterrupt: 

In [ ]:
from universal_pytorch import test_uap

test_uap.main(model, data_dir + '/val', img_list, batch_size, gpu)